In [10]:
# Utils

import os
import cv2
import json
import numpy as np
from PIL import Image
import math 
from random import randrange, sample

def getMaxContour(img_file_path):
     # Read input
     img = cv2.imread(img_file_path, cv2.IMREAD_GRAYSCALE)
     # Threshold
     ret,thresh = cv2.threshold(img,1,255,cv2.THRESH_BINARY)
     # Max contour
     contours,hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
     return max(contours, key = cv2.contourArea)

def apply_re(img, mask_coordinates, bb_coordinates, output_path, image_idx):
    img_h, img_w, c = img.shape

    mask_left, mask_top, mask_right, mask_bot = mask_coordinates
    bb_left, bb_top, bb_right, bb_bot = bb_coordinates

    # generate mask
    mask = np.zeros((img_h, img_w, 1), dtype="uint8")
    cv2.rectangle(mask, (mask_left, mask_top), (mask_right, mask_bot), 255, -2)

    # occlude image
    occ_image = img*(1. - mask)

    # generate bb and mask border
    img_cnt = img.copy()
    alpha = 0.4

    # mask rectangle and border
    cv2.rectangle(img_cnt, (mask_left, mask_top), (mask_right, mask_bot), (0,0,219,255), -2)
    img_cnt = cv2.addWeighted(img_cnt, alpha, img, 1 - alpha, 0)
    cv2.rectangle(img_cnt, (mask_left, mask_top), (mask_right, mask_bot), (0,0,219,255), 1)
    # bb border 
    cv2.rectangle(img_cnt, (bb_left, bb_top), (bb_right, bb_bot), (219,0,0,255), 1)
        
    output_img_path = os.path.join(output_path, '%02d.png'%image_idx)
    output_mask_path = os.path.join(output_path, '%02d_mask.png'%image_idx)
    output_contour_path = os.path.join(output_path, '%02d_contour.png'%image_idx)

    cv2.imwrite(output_img_path, occ_image)
    cv2.imwrite(output_mask_path, mask)
    cv2.imwrite(output_contour_path, img_cnt)

def get_re_coordinates(bb_coordinates, args):

    bb_left, bb_top, bb_right, bb_bot = bb_coordinates
    
    bb_h = bb_bot-bb_top
    bb_w = bb_right-bb_left

    bb_area = bb_h*bb_w
    mask_area = bb_area*args["percentage"]/100
    mask_w = math.floor(math.sqrt(mask_area))
    mask_h = mask_w

    if mask_h>bb_h:
        mask_h = bb_h
        mask_w = math.floor(mask_area/mask_h)

    if mask_w>bb_w:
        mask_w = bb_w
        mask_h = math.floor(mask_area/mask_w)

    mask_top = bb_top+math.floor(bb_h/2-mask_h/2)
    mask_left = bb_left+math.floor(bb_w/2-mask_w/2)

    if args["location"].lower() == "random":
        if(bb_top != bb_bot-mask_h):
            mask_top = randrange(bb_top, bb_bot-mask_h)
        if(bb_left != bb_right-mask_w):
            mask_left = randrange(bb_left, bb_right-mask_w)

    return [mask_left, mask_top, mask_left+mask_w, mask_top+mask_h]

def loop_ShapeNet_dataset(function_process_images, renders_path, output_path, taxonomy_file, args=None):
    
    class_id = "03001627"

    taxonomies = []
    with open(taxonomy_file, encoding='utf-8') as file:
        taxonomies = json.loads(file.read())

    taxonomies = {t['taxonomy_id']: t for t in taxonomies}
    model_ids = taxonomies[class_id]["test"]

    for sample_idx, sample_name in enumerate(model_ids):

        print("=== %04d/%04d === %s" %(sample_idx,len(model_ids),sample_name))

        output_sample_path = os.path.join(output_path, class_id, sample_name, "rendering")

        if not os.path.exists(output_sample_path):
            os.makedirs(output_sample_path)
            
        # Get file list of rendering images
        img_file_path = renders_path % (class_id, sample_name, 0)
        rendering_folder = os.path.dirname(img_file_path)
        n_renders = len(os.listdir(rendering_folder))

        for image_idx in range(n_renders):
            img_file_path = renders_path % (class_id, sample_name, image_idx)
            
            if os.path.exists(img_file_path):
                function_process_images(img_file_path, output_sample_path, image_idx, args)

### Cropping and resizing ShapeNet

In [12]:
taxonomy_file = "/home/davide/Desktop/Dissertation/Models/Pix2Vox/datasets/ShapeNet_Chair.json"
renders_path = '/home/davide/Desktop/Dissertation/Data/ShapeNetRendering/%s/%s/rendering/%02d.png'
output_path = '/home/davide/Desktop/Dissertation/Data/Tests/CropShapeNet/'

args = {
    "border":0,
    "new_size": (224,224)
}

def resizeAndPad(img, size, border=10, padColor=0):

    h, w = img.shape[:2]
    sh, sw = size
    interp = cv2.INTER_CUBIC

    # aspect ratio of image
    aspect = w/h  # if on Python 2, you might need to cast as a float: float(w)/h

    # compute scaling and pad sizing
    if aspect > 1: # horizontal image
        new_w = sw-border*2
        new_h = np.round(new_w/aspect).astype(int)
        pad_vert = (sh-new_h)/2
        pad_top, pad_bot = np.floor(pad_vert).astype(int), np.ceil(pad_vert).astype(int)
        pad_left, pad_right = border, border
    elif aspect < 1: # vertical image
        new_h = sh-border*2
        new_w = np.round(new_h*aspect).astype(int)
        pad_horz = (sw-new_w)/2
        pad_left, pad_right = np.floor(pad_horz).astype(int), np.ceil(pad_horz).astype(int)
        pad_top, pad_bot = border, border
    else: # square image
        new_h, new_w = sh, sw
        pad_left, pad_right, pad_top, pad_bot = border, border, border, border

    # set pad color
    if len(img.shape) == 3 and not isinstance(padColor, (list, tuple, np.ndarray)): # color image but only one color provided
        padColor = [padColor]*3

    # scale and pad
    scaled_img = cv2.resize(img, (new_w, new_h), interpolation=interp)
    scaled_img = cv2.copyMakeBorder(scaled_img, pad_top, pad_bot, pad_left, pad_right, borderType=cv2.BORDER_CONSTANT, value=padColor)

    return scaled_img

def cropAndResize(input, output_path, image_idx, args):

    border = args["border"]
    new_size = args["new_size"]

    image = Image.open(input)
    left, top, right, bottom = image.getbbox()
    rendering_image = cv2.imread(input, cv2.IMREAD_UNCHANGED)

    cropped = rendering_image[top:bottom, left:right, :]
    result = resizeAndPad(cropped, new_size, border)

    output_img_path = os.path.join(output_path, '%02d.png'%image_idx)
    cv2.imwrite(output_img_path, result)

loop_ShapeNet_dataset(cropAndResize, renders_path, output_path, taxonomy_file, args)

=== 0000/1357 === 3b3a9f4e3aa9f2f4d39a194653571dfc
=== 0001/1357 === a691eee4545ce2fade94aad0562ac2e
=== 0002/1357 === a58f8f1bd61094b3ff2c92c2a4f65876
=== 0003/1357 === 27c00ec2b6ec279958e80128fd34c2b1
=== 0004/1357 === 484f0070df7d5375492d9da2668ec34c
=== 0005/1357 === 1e678fabd0622a1119fb4103277a6b93
=== 0006/1357 === 4231883e92a3c1a21c62d11641ffbd35
=== 0007/1357 === 8f2cc8ff68f3208ec935dd3bb5739fc
=== 0008/1357 === 717e28c855c935c94d2d89cc1fd36fca
=== 0009/1357 === 5ca11d3f35d870e1b724bccd568c5fc1
=== 0010/1357 === 701551efa4f1f4fe3e478b26cb10ebe4
=== 0011/1357 === d0e24e315e67bff2cd80b6234fc6097f
=== 0012/1357 === beb4c42cfa1c3b282811d30bba54859
=== 0013/1357 === 7bdc0aaca74cca86593ebeeedbff73b
=== 0014/1357 === 4960144353a6dd944c5483fab67d9a50
=== 0015/1357 === 4c3f655507cc6aecdbe9c90af547c85d
=== 0016/1357 === 53d88f50e3abc2888dcbe86402c7c15
=== 0017/1357 === b46ad21b7126049842ca7cc070f21ed3
=== 0018/1357 === 383a9a2c39332f7f50eb11cb3709022
=== 0019/1357 === d7db1353551d341f149

### IRE

In [7]:
def ire(img_file_path, output_path, image_idx, args):

    img = cv2.imread(img_file_path, cv2.IMREAD_UNCHANGED)
    
    img_h, img_w, c = img.shape
    bb_coordinates = [0, 0, img_h-1, img_w-1]

    mask_coordinates = get_re_coordinates(bb_coordinates, args)
    
    apply_re(img, mask_coordinates, bb_coordinates, output_path, image_idx)

taxonomy_file = "/home/davide/Desktop/Dissertation/Models/Pix2Vox/datasets/ShapeNet_Chair.json"
renders_path = '/home/davide/Desktop/Dissertation/Data/ShapeNetRendering/%s/%s/rendering/%02d.png'
percentages = [15,25,50]
locations = ["Center", "Random"]

for p in percentages:

    for l in locations:
        output_path = f'/home/davide/Desktop/Dissertation/Data/OccludedShapeNet/Occluded/IRE_{l}_{p}/'

        args = {
            "location": l,
            "percentage": p
        }

        loop_ShapeNet_dataset(ire, renders_path, output_path, taxonomy_file, args)

=== 0000/1357 === 3b3a9f4e3aa9f2f4d39a194653571dfc
=== 0001/1357 === a691eee4545ce2fade94aad0562ac2e
=== 0002/1357 === a58f8f1bd61094b3ff2c92c2a4f65876
=== 0003/1357 === 27c00ec2b6ec279958e80128fd34c2b1
=== 0004/1357 === 484f0070df7d5375492d9da2668ec34c
=== 0005/1357 === 1e678fabd0622a1119fb4103277a6b93
=== 0006/1357 === 4231883e92a3c1a21c62d11641ffbd35
=== 0007/1357 === 8f2cc8ff68f3208ec935dd3bb5739fc
=== 0008/1357 === 717e28c855c935c94d2d89cc1fd36fca
=== 0009/1357 === 5ca11d3f35d870e1b724bccd568c5fc1
=== 0010/1357 === 701551efa4f1f4fe3e478b26cb10ebe4
=== 0011/1357 === d0e24e315e67bff2cd80b6234fc6097f
=== 0012/1357 === beb4c42cfa1c3b282811d30bba54859
=== 0013/1357 === 7bdc0aaca74cca86593ebeeedbff73b
=== 0014/1357 === 4960144353a6dd944c5483fab67d9a50
=== 0015/1357 === 4c3f655507cc6aecdbe9c90af547c85d
=== 0016/1357 === 53d88f50e3abc2888dcbe86402c7c15
=== 0017/1357 === b46ad21b7126049842ca7cc070f21ed3
=== 0018/1357 === 383a9a2c39332f7f50eb11cb3709022
=== 0019/1357 === d7db1353551d341f149

### ORE

In [10]:
def ore(img_file_path, output_path, image_idx, args):

    img = cv2.imread(img_file_path, cv2.IMREAD_UNCHANGED)

    cnt = getMaxContour(img_file_path)
    left, top, bb_w, bb_h = cv2.boundingRect(cnt)
    bb_coordinates = [left, top, left+bb_w, top+bb_h]
    
    mask_coordinates = get_re_coordinates(bb_coordinates, args)
    
    apply_re(img, mask_coordinates, bb_coordinates, output_path, image_idx)


taxonomy_file = "/home/davide/Desktop/Dissertation/Models/Pix2Vox/datasets/ShapeNet_Test.json"
taxonomy_file = "/home/davide/Desktop/Dissertation/Models/Pix2Vox/datasets/ShapeNet_Chair.json"
renders_path = '/home/davide/Desktop/Dissertation/Data/ShapeNetRendering/%s/%s/rendering/%02d.png'
percentages = [15,25,50]
locations = ["Center", "Random"]

for p in percentages:

    for l in locations:
        output_path = f'/home/davide/Desktop/Dissertation/Data/OccludedShapeNet/Occluded/ORE_{l}_{p}/'

        args = {
            "location": l,
            "percentage": p
        }

        loop_ShapeNet_dataset(ore, renders_path, output_path, taxonomy_file, args)


=== 0000/1357 === 3b3a9f4e3aa9f2f4d39a194653571dfc
=== 0001/1357 === a691eee4545ce2fade94aad0562ac2e
=== 0002/1357 === a58f8f1bd61094b3ff2c92c2a4f65876
=== 0003/1357 === 27c00ec2b6ec279958e80128fd34c2b1
=== 0004/1357 === 484f0070df7d5375492d9da2668ec34c
=== 0005/1357 === 1e678fabd0622a1119fb4103277a6b93
=== 0006/1357 === 4231883e92a3c1a21c62d11641ffbd35
=== 0007/1357 === 8f2cc8ff68f3208ec935dd3bb5739fc
=== 0008/1357 === 717e28c855c935c94d2d89cc1fd36fca
=== 0009/1357 === 5ca11d3f35d870e1b724bccd568c5fc1
=== 0010/1357 === 701551efa4f1f4fe3e478b26cb10ebe4
=== 0011/1357 === d0e24e315e67bff2cd80b6234fc6097f
=== 0012/1357 === beb4c42cfa1c3b282811d30bba54859
=== 0013/1357 === 7bdc0aaca74cca86593ebeeedbff73b
=== 0014/1357 === 4960144353a6dd944c5483fab67d9a50
=== 0015/1357 === 4c3f655507cc6aecdbe9c90af547c85d
=== 0016/1357 === 53d88f50e3abc2888dcbe86402c7c15
=== 0017/1357 === b46ad21b7126049842ca7cc070f21ed3
=== 0018/1357 === 383a9a2c39332f7f50eb11cb3709022
=== 0019/1357 === d7db1353551d341f149

### MORE

In [8]:
import math 

def more(img_file_path, output_path, image_idx, args):

    img = cv2.imread(img_file_path, cv2.IMREAD_UNCHANGED)

    cnt = getMaxContour(img_file_path)
    rect_bb = cv2.minAreaRect(cnt)
    box_bb = cv2.boxPoints(rect_bb)
    box_bb = np.int0(box_bb)

    # rect [0] -> center point
    # rect [1] -> w,h
    # rect [2] -> angle
    bb_center, bb_shape, bb_angle = rect_bb

    bb_w = bb_shape[0]
    bb_h = bb_shape[1]

    # bb_area = cv2.contourArea(cnt)
    bb_area = bb_h*bb_w
    mask_area = bb_area*args["percentage"]/100
    mask_w = math.floor(math.sqrt(mask_area))
    mask_h = mask_w

    if mask_h>bb_h:
        mask_h = bb_h
        mask_w = math.floor(mask_area/mask_h)

    if mask_w>bb_w:
        mask_w = bb_w
        mask_h = math.floor(mask_area/mask_w)

    rect_mask = (bb_center, (mask_w, mask_h), bb_angle)
    box_mask = cv2.boxPoints(rect_mask)
    box_mask = np.int0(box_mask)

    img_h, img_w, c = img.shape
    mask = np.zeros((img_h, img_w, 1), dtype="uint8")

    cv2.drawContours(mask, [box_mask], 0, 255, -2)

    occ_image = img*(1. - mask)

    output_img_path = os.path.join(output_path, '%02d.png'%image_idx)
    output_mask_path = os.path.join(output_path, '%02d_mask.png'%image_idx)
    output_contour_path = os.path.join(output_path, '%02d_contour.png'%image_idx)

    cv2.imwrite(output_img_path, occ_image)
    cv2.imwrite(output_mask_path, mask)

    cv2.drawContours(img, [box_mask], 0, (0,0,219,255), 1)
    cv2.drawContours(img, [box_bb], 0, (219,0,0,255), 1)

    temp_img = img.copy()
    alpha = 0.4

    cv2.drawContours(img, [box_mask], 0, (0,0,219,255), -2)

    img = cv2.addWeighted(temp_img, alpha, img, 1 - alpha, 0)
    cv2.imwrite(output_contour_path, img)

    return mask, occ_image

# taxonomy_file = "/home/davide/Desktop/Dissertation/Models/Pix2Vox/datasets/ShapeNet_Test.json"
taxonomy_file = "/home/davide/Desktop/Dissertation/Models/Pix2Vox/datasets/ShapeNet_Chair.json"
renders_path = '/home/davide/Desktop/Dissertation/Data/ShapeNetRendering/%s/%s/rendering/%02d.png'
percentages = [15,25,50]

location = "Center"

for p in percentages:

    output_path = f'/home/davide/Desktop/Dissertation/Data/OccludedShapeNet/Occluded/MORE_{location}_{p}/'

    args = {
        "location": location,
        "percentage": p
    }

    loop_ShapeNet_dataset(more, renders_path, output_path, taxonomy_file, args)

=== 0000/1357 === 3b3a9f4e3aa9f2f4d39a194653571dfc
=== 0001/1357 === a691eee4545ce2fade94aad0562ac2e
=== 0002/1357 === a58f8f1bd61094b3ff2c92c2a4f65876
=== 0003/1357 === 27c00ec2b6ec279958e80128fd34c2b1
=== 0004/1357 === 484f0070df7d5375492d9da2668ec34c
=== 0005/1357 === 1e678fabd0622a1119fb4103277a6b93
=== 0006/1357 === 4231883e92a3c1a21c62d11641ffbd35
=== 0007/1357 === 8f2cc8ff68f3208ec935dd3bb5739fc
=== 0008/1357 === 717e28c855c935c94d2d89cc1fd36fca
=== 0009/1357 === 5ca11d3f35d870e1b724bccd568c5fc1
=== 0010/1357 === 701551efa4f1f4fe3e478b26cb10ebe4
=== 0011/1357 === d0e24e315e67bff2cd80b6234fc6097f
=== 0012/1357 === beb4c42cfa1c3b282811d30bba54859
=== 0013/1357 === 7bdc0aaca74cca86593ebeeedbff73b
=== 0014/1357 === 4960144353a6dd944c5483fab67d9a50
=== 0015/1357 === 4c3f655507cc6aecdbe9c90af547c85d
=== 0016/1357 === 53d88f50e3abc2888dcbe86402c7c15
=== 0017/1357 === b46ad21b7126049842ca7cc070f21ed3
=== 0018/1357 === 383a9a2c39332f7f50eb11cb3709022
=== 0019/1357 === d7db1353551d341f149

### Irregular

In [18]:
# Code from the implementation of Palette inpainting
# https://github.com/Janspiry/Palette-Image-to-Image-Diffusion-Models/tree/main

def random_irregular_mask(img_shape,
                          num_vertices=(4, 8),
                          max_angle=4,
                          length_range=(10, 100),
                          brush_width=(10, 40),
                          dtype='uint8'):
    """Generate random irregular masks.
    This is a modified version of free-form mask implemented in
    'brush_stroke_mask'.
    We prefer to use `uint8` as the data type of masks, which may be different
    from other codes in the community.
    TODO: Rewrite the implementation of this function.
    Args:
        img_shape (tuple[int]): Size of the image.
        num_vertices (int | tuple[int]): Min and max number of vertices. If
            only give an integer, we will fix the number of vertices.
            Default: (4, 8).
        max_angle (float): Max value of angle at each vertex. Default 4.0.
        length_range (int | tuple[int]): (min_length, max_length). If only give
            an integer, we will fix the length of brush. Default: (10, 100).
        brush_width (int | tuple[int]): (min_width, max_width). If only give
            an integer, we will fix the width of brush. Default: (10, 40).
        dtype (str): Indicate the data type of returned masks. Default: 'uint8'
    Returns:
        numpy.ndarray: Mask in the shape of (h, w, 1).
    """

    h, w = img_shape[:2]

    mask = np.zeros((h, w), dtype=dtype)
    if isinstance(length_range, int):
        min_length, max_length = length_range, length_range + 1
    elif isinstance(length_range, tuple):
        min_length, max_length = length_range
    else:
        raise TypeError('The type of length_range should be int'
                        f'or tuple[int], but got type: {length_range}')
    if isinstance(num_vertices, int):
        min_num_vertices, max_num_vertices = num_vertices, num_vertices + 1
    elif isinstance(num_vertices, tuple):
        min_num_vertices, max_num_vertices = num_vertices
    else:
        raise TypeError('The type of num_vertices should be int'
                        f'or tuple[int], but got type: {num_vertices}')

    if isinstance(brush_width, int):
        min_brush_width, max_brush_width = brush_width, brush_width + 1
    elif isinstance(brush_width, tuple):
        min_brush_width, max_brush_width = brush_width
    else:
        raise TypeError('The type of brush_width should be int'
                        f'or tuple[int], but got type: {brush_width}')

    num_v = np.random.randint(min_num_vertices, max_num_vertices)

    for i in range(num_v):
        start_x = np.random.randint(w)
        start_y = np.random.randint(h)
        # from the start point, randomly setlect n \in [1, 6] directions.
        direction_num = np.random.randint(1, 6)
        angle_list = np.random.randint(0, max_angle, size=direction_num)
        length_list = np.random.randint(
            min_length, max_length, size=direction_num)
        brush_width_list = np.random.randint(
            min_brush_width, max_brush_width, size=direction_num)
        for direct_n in range(direction_num):
            angle = 0.01 + angle_list[direct_n]
            if i % 2 == 0:
                angle = 2 * math.pi - angle
            length = length_list[direct_n]
            brush_w = brush_width_list[direct_n]
            # compute end point according to the random angle
            end_x = (start_x + length * np.sin(angle)).astype(np.int32)
            end_y = (start_y + length * np.cos(angle)).astype(np.int32)

            cv2.line(mask, (start_y, start_x), (end_y, end_x), 1, brush_w)
            start_x, start_y = end_x, end_y
    mask = np.expand_dims(mask, axis=2)

    return mask


def get_irregular_mask(img_shape, area_ratio_range=(0.15, 0.5), **kwargs):
    """Get irregular mask with the constraints in mask ratio
    Args:
        img_shape (tuple[int]): Size of the image.
        area_ratio_range (tuple(float)): Contain the minimum and maximum area
        ratio. Default: (0.15, 0.5).
    Returns:
        numpy.ndarray: Mask in the shape of (h, w, 1).
    """

    mask = random_irregular_mask(img_shape, **kwargs)
    min_ratio, max_ratio = area_ratio_range

    while not min_ratio < (np.sum(mask) /
                           (img_shape[0] * img_shape[1])) < max_ratio:
        mask = random_irregular_mask(img_shape, **kwargs)

    return mask

def addRandomIrregularMask(img_file_path, output_path, image_idx, args):
    ratio = args["erasing_ratios"]

    img = Image.open(img_file_path)
    img_h, img_w = img.size

    b_l, b_t, b_r, b_b = img.getbbox()
    b_h = b_b - b_t 
    b_w = b_r - b_l
    
    # mask_erasing, occ_image, boxed_image = masking_function(img_file_path, percentage)
    random_mask = get_irregular_mask((b_h,b_w), ratio)


    mask = np.zeros((img_h, img_w, 1), dtype="uint8")
    mask[b_t:b_b, b_l:b_r, :] = random_mask

    image = cv2.imread(img_file_path, cv2.IMREAD_UNCHANGED)
    occ_image = image*(1. - mask)

    ret, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)
    
    save_mask_erasing = os.path.join(output_path, '%02d_mask.png'%image_idx)
    save_occ_image = os.path.join(output_path, '%02d.png'%image_idx)
    
    cv2.imwrite(save_occ_image, occ_image)
    cv2.imwrite(save_mask_erasing, mask)

In [20]:
taxonomy_file = "/home/davide/Desktop/Project/Models/Pix2Vox/datasets/ShapeNet_Chair.json"
taxonomy_file = "/home/davide/Desktop/Project/Models/Pix2Vox/datasets/ShapeNet_Test.json"
renders_path = '/home/davide/Desktop/Project/Data/ShapeNetRendering/%s/%s/rendering/%02d.png'
output_path = "/home/davide/Desktop/Dissertation/Data/tests/"

# erasing_percentages = [15, 25, 50]
dataset_name = "OccShapeNet"

erasing_ratios = [(.20,.30), (.40,.50)]
erasing_ratios = [(.40,.80)]

for ratio in erasing_ratios:

    name = f"Irregular_{int(ratio[0]*100)}_{int(ratio[1]*100)}"
    output_dataset_path = os.path.join(output_path, name)

    if not os.path.exists(output_dataset_path):
        os.makedirs(output_dataset_path)
            

    args = {
        "erasing_ratios": ratio,
    }

    print(f"-- Generating {name}")

    loop_ShapeNet_dataset(addRandomIrregularMask, renders_path, output_dataset_path, taxonomy_file, args)

-- Generating Irregular_40_80
=== 0000/0003 === 3b3a9f4e3aa9f2f4d39a194653571dfc
=== 0001/0003 === be4c88a130e622a21961e650f3cfa396
=== 0002/0003 === 701551efa4f1f4fe3e478b26cb10ebe4


## Sample datasets

In [4]:
import torchvision
from torchvision.io import read_image
from torchvision.utils import make_grid
from random import sample

def sample_dataset(renders_path, taxonomy_file, n_models=4,n_samples=1, nrow=2, padding=0):
    
    class_id = "03001627"

    taxonomies = []
    with open(taxonomy_file, encoding='utf-8') as file:
        taxonomies = json.loads(file.read())

    taxonomies = {t['taxonomy_id']: t for t in taxonomies}

    model_ids = taxonomies[class_id]["test"]
    sample_model_ids = sample(model_ids, n_models)
    # print(sample_model_ids)

    images = []

    for sample_idx, sample_name in enumerate(sample_model_ids):

        # print("=== %04d/%04d === %s" %(sample_idx,len(sample_model_ids),sample_name))
            
        # Get file list of rendering images
        img_file_path = renders_path % (class_id, sample_name, 0)
        rendering_folder = os.path.dirname(img_file_path)
        n_renders = len(os.listdir(rendering_folder))
        sample_img_ids = sample(range(20), n_samples)
        # print(sample_img_ids)

        for image_idx in sample_img_ids:
            img_file_path = renders_path % (class_id, sample_name, image_idx)

            if os.path.exists(img_file_path):
                images.append(read_image(img_file_path))

    # print(len(images))
    Grid = make_grid(images, nrow=nrow, padding=padding,pad_value=179)
    # display result
    img = torchvision.transforms.ToPILImage()(Grid)
    return img

In [11]:
datasets_path = "/home/davide/Desktop/Dissertation/Data/Tests/Occluded/"
save_path = "infoLoss/Occluded"
if not os.path.exists(save_path):
    os.makedirs(save_path)
taxonomy_file = "/home/davide/Desktop/Dissertation/Models/Pix2Vox/datasets/ShapeNet_Chair.json"

for dataset in os.listdir(datasets_path):
    renders_path = os.path.join(datasets_path,dataset,"%s/%s/rendering/%02d_contour.png")
    img = sample_dataset(renders_path, taxonomy_file, n_models=4,n_samples=1, nrow=2, padding=1)
    img.save(os.path.join(save_path, "samples_"+dataset+".png"))

    print(dataset+" sampled")

=== 0000/0004 === a598b87a9e37a438617c72672c86a49
=== 0001/0004 === 735df13871e3ab4edf6cfab91d65bb91
=== 0002/0004 === e779cf261cf04a77acd8c40fddcf9ca
=== 0003/0004 === c236deaff8c6fb0d29c9a7a92b0a566d
4
IRE_Center_25 sampled
=== 0000/0004 === 41d9bd662687cf503ca22f17e86bab24
=== 0001/0004 === 6678f63c9b584a549d9e5580ae9f8738
=== 0002/0004 === 7f3562432f02b8a897d3ea932731fd42
=== 0003/0004 === d7867d215f52107ba5e8cf3aa1686d66
4
ORE_Center_15 sampled
=== 0000/0004 === bf7e8e0dc4f4038cc2567be77cb7ab45
=== 0001/0004 === 3ff879e891701079ea07bd8096c66582
=== 0002/0004 === 570527a5388977c878ba201e6a414b21
=== 0003/0004 === bb90094030f369e4305a3b2fd9173d6f
4
MORE_Center_50 sampled
=== 0000/0004 === bca8b73b32108ad6d42ec7e303174a87
=== 0001/0004 === d7dfcd99197d6ae7b6dbfb7eb145012e
=== 0002/0004 === 2f4fe1db48f0cac1db573653825dd010
=== 0003/0004 === 7fd10f44a20506ec1b17743c18fb63dc
4
IRE_Center_15 sampled
=== 0000/0004 === 6ecbaaaebb241ec34b3c42e318f3affc
=== 0001/0004 === 4a45de600363af6bd5cb